In [1]:

## Parameters specific to where your folders are and your data
parameter_file = 'params/params.yaml'
import yaml
import sys

with open(parameter_file,'r') as f:
    doc = yaml.load(f)

#p = dic2obj(**doc)

data_folder = doc['data_folder']
tissues = doc['tissues'].split(',')
sys.path.append(doc['pipeline_path'])
ref_fa = doc['ref_fa']
annotation=doc['annotation']


tss_annotation = doc['tss_annotation']

import os
import sys
import pandas as pd
import matplotlib
import seaborn as sns
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from itertools import product
import glob
import re
from matplotlib_venn import venn2
from matplotlib import rcParams
import inspect
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
sys.setrecursionlimit(3000)
%load_ext autoreload
%autoreload 2
rcParams['figure.figsize'] = 8, 6
import tqdm
### notebook specific configuration ###
from os.path import basename
#mpl.style.use('ggplot')
mpl.style.use('fivethirtyeight')
from cycler import cycler
mpl.rcParams['axes.prop_cycle'] = cycler(color='bgrcmyk')

from Homer import *

print('Number of tissues: ',len(tissues))

('Number of tissues: ', 15)


In [2]:
tissues

['BMDM1hKLA',
 'BMDMwt',
 'Brain',
 'FemaleReproductive',
 'Heart',
 'Kidney',
 'Liver',
 'Lung',
 'MiscOrgans',
 'Muscle',
 'Pancreas',
 'Skin',
 'Spleen',
 'Total',
 'CHO']

In [6]:
merged_expr = pd.read_csv("Results/merged/samples.merge.peaksexpression",sep="\t",index_col=0)
merged_expr.head()

,f04_peaks/Brain_START2_and_Brain_STARTinput1,f04_peaks/Brain_GROCap1_and_Brain_GRO1,f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_STARTinput1,f04_peaks/Kidney_START1_and_Kidney_STARTinput1,f04_peaks/Muscle_START2_and_Muscle_STARTinput1,f04_peaks/Heart_START1_and_Heart_STARTinput1,f04_peaks/Lung_GROCap1_and_Lung_GRO1,f04_peaks/Liver_START2_and_Liver_STARTinput1,f04_peaks/MiscOrgans_START2_and_MiscOrgans_STARTinput1,f04_peaks/Muscle_START1_and_Muscle_STARTinput1,...,f04_peaks/Brain_START1_and_Brain_STARTinput2,f04_peaks/FemaleReproductive_START2_and_FemaleReproductive_STARTinput1,f04_peaks/Pancreas_START2_and_Pancreas_STARTinput1,f04_peaks/Brain_START3_and_Brain_STARTinput1,f04_peaks/Kidney_GROCap1_and_Kidney_GRO1,f04_peaks/Brain_START1_and_Brain_STARTinput1,f04_peaks/CHO_GROCap1_and_CHO_GRO1,f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1,f04_peaks/Heart_START2_and_Heart_STARTinput1,f04_peaks/Lung_START1_and_Lung_STARTinput1
0,15.5,5.12,0.0,26.50,22.50,2.34,0.0,0.0,9.37,1.93,...,23.00,16.10,10.10,11.60,1.05,23.00,3.12,4.7,0.0,14.4
1,9.6,50.80,0.0,4.09,7.78,3.68,0.0,0.0,6.30,5.78,...,5.25,5.85,4.77,2.48,103.00,5.25,46.00,65.6,0.0,10.7
2,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,1.15,0.00,0.00,0.0,0.0,0.0
3,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.00,0.00,...,0.00,0.00,0.00,0.00,1.94,0.00,0.00,0.0,0.0,0.0
4,0.0,0.00,0.0,0.00,0.00,0.00,0.0,2.5,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.79,0.0,0.0,0.0


In [9]:
tissue_map = dict()
tissue_map_short = dict() #Remove the f04_peals
for i in merged_expr.columns.values:
    for t in tissues:
        if t in i:
            if i in tissue_map:
                print("Another tissue name was found.", i)
                break
            tissue_map[i] = t
            tissue_map_short[os.path.basename(i)] = t

In [8]:
tissue_map

{'f04_peaks/BMDMwt_GROCap1_and_BMDMwt_GRO1': 'BMDMwt',
 'f04_peaks/Brain_GROCap1_and_Brain_GRO1': 'Brain',
 'f04_peaks/Brain_START1_and_Brain_STARTinput1': 'Brain',
 'f04_peaks/Brain_START1_and_Brain_STARTinput2': 'Brain',
 'f04_peaks/Brain_START2_and_Brain_STARTinput1': 'Brain',
 'f04_peaks/Brain_START2_and_Brain_STARTinput2': 'Brain',
 'f04_peaks/Brain_START3_and_Brain_STARTinput1': 'Brain',
 'f04_peaks/Brain_START3_and_Brain_STARTinput2': 'Brain',
 'f04_peaks/CHO_GROCap1_and_CHO_GRO1': 'CHO',
 'f04_peaks/CHO_GROCap1_and_CHO_GRO2': 'CHO',
 'f04_peaks/CHO_GROCap2_and_CHO_GRO1': 'CHO',
 'f04_peaks/CHO_GROCap2_and_CHO_GRO2': 'CHO',
 'f04_peaks/CHO_START1_and_CHO_STARTinput1': 'CHO',
 'f04_peaks/FemaleReproductive_START1_and_FemaleReproductive_STARTinput1': 'FemaleReproductive',
 'f04_peaks/FemaleReproductive_START2_and_FemaleReproductive_STARTinput1': 'FemaleReproductive',
 'f04_peaks/Heart_START1_and_Heart_STARTinput1': 'Heart',
 'f04_peaks/Heart_START2_and_Heart_STARTinput1': 'Heart',

In [10]:
tissue_map_short

{'BMDMwt_GROCap1_and_BMDMwt_GRO1': 'BMDMwt',
 'Brain_GROCap1_and_Brain_GRO1': 'Brain',
 'Brain_START1_and_Brain_STARTinput1': 'Brain',
 'Brain_START1_and_Brain_STARTinput2': 'Brain',
 'Brain_START2_and_Brain_STARTinput1': 'Brain',
 'Brain_START2_and_Brain_STARTinput2': 'Brain',
 'Brain_START3_and_Brain_STARTinput1': 'Brain',
 'Brain_START3_and_Brain_STARTinput2': 'Brain',
 'CHO_GROCap1_and_CHO_GRO1': 'CHO',
 'CHO_GROCap1_and_CHO_GRO2': 'CHO',
 'CHO_GROCap2_and_CHO_GRO1': 'CHO',
 'CHO_GROCap2_and_CHO_GRO2': 'CHO',
 'CHO_START1_and_CHO_STARTinput1': 'CHO',
 'FemaleReproductive_START1_and_FemaleReproductive_STARTinput1': 'FemaleReproductive',
 'FemaleReproductive_START2_and_FemaleReproductive_STARTinput1': 'FemaleReproductive',
 'Heart_START1_and_Heart_STARTinput1': 'Heart',
 'Heart_START2_and_Heart_STARTinput1': 'Heart',
 'Kidney_GROCap1_and_Kidney_GRO1': 'Kidney',
 'Kidney_START1_and_Kidney_STARTinput1': 'Kidney',
 'Kidney_START2_and_Kidney_STARTinput1': 'Kidney',
 'Liver_GROCap1_and_Li

In [12]:
pickle.dump([tissue_map,tissue_map_short],open("Results/tissue_map.p","wb"))